In [16]:
import pickle
import random

import numpy as np

matches_df = pickle.load(open("database/matches_df.pck", "rb"))

In [17]:
winner_list = []
market_val_diff = []
goals_diff = []
for _, row in matches_df.iterrows():
    if row["Team Home ID"] == row["Winner Team ID"]:
        winner_list.append("Home")
    elif row["Team Away ID"] == row["Winner Team ID"]:
        winner_list.append("Away")
    elif row["Winner Team ID"] == 0:
        winner_list.append("Draw")
    market_val_diff.append(row["Market Value Home"] - row["Market Value Away"])
    goals_diff.append(row["Goals Home"] - row["Goals Away"])
matches_df["Result"] = winner_list
matches_df["Market Value Difference"] = market_val_diff
matches_df["Goals Difference"] = goals_diff

In [18]:
import asyncio
import nest_asyncio
from database import dbconn
import importlib

importlib.reload(dbconn)

nest_asyncio.apply()

loop = asyncio.get_event_loop()



In [19]:
import pandas as pd

dict_team_data = dict()

for team_id in set(matches_df["Team Home ID"]):
    team_data_matches_home, team_data_matches_away = loop.run_until_complete(dbconn.get_matches_by_team(team_id))
    team_data_matches = [[x.id, x.goalsHome, x.goalsAway, x.winnerTeamId] for x in team_data_matches_home] + [[x.id, x.goalsHome, x.goalsAway, x.winnerTeamId] for x in team_data_matches_away]
    team_data_matches = sorted(team_data_matches, key=lambda x: x[0])
    for i in range(len(team_data_matches)):
        if team_data_matches[i][3] == team_id:
            team_data_matches[i][3] = "Won"
        elif team_data_matches[i][3] == None:
            team_data_matches[i][3] = "Draw"
        else:
            team_data_matches[i][3] = "Lost"
    dict_team_data[team_id] = pd.DataFrame(team_data_matches)

In [20]:
matches_form = []
for _, row in matches_df.iterrows():
    home_form = dict_team_data[row["Team Home ID"]]
    home_form = home_form[home_form[0] < row["Match ID"]].iloc[-5:]
    home_form_5 = list(home_form[3])
    for i in range(5 - len(home_form_5)):
        home_form_5 = ["Draw"] + home_form_5
    away_form = dict_team_data[row["Team Away ID"]]
    away_form = away_form[away_form[0] < row["Match ID"]].iloc[-5:]
    away_form_5 = list(away_form[3])
    for i in range(5 - len(away_form_5)):
        away_form_5 = ["Draw"] + away_form_5
    matches_form.append(home_form_5 + away_form_5)

In [21]:
columns_added = ["Home_Pre" + str(x) for x in range(5, 0, -1)] + ["Away_Pre" + str(x) for x in range(5, 0, -1)]
matches_df = pd.concat([matches_df, pd.DataFrame(matches_form, columns=columns_added)], axis=1)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *

res_enc = LabelEncoder()
form_enc = LabelEncoder()
matches_df["Result"] = res_enc.fit_transform(matches_df["Result"])
form_enc.fit(matches_df["Home_Pre5"])
matches_df["Home_Pre5"] = form_enc.transform(matches_df["Home_Pre5"])
for column in columns_added[1:]:
    matches_df[column] = form_enc.transform(matches_df[column])
X_train, X_test, y_train, y_test = train_test_split(matches_df.drop(["Goals Home", "Goals Away", "Goals Difference", "Result", "Winner Team ID", "Match ID", "Team Home ID", "Team Away ID"], axis=1), matches_df["Result"], random_state=42, test_size=0.2)

In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *

clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:591: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:600: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match

0.48366013071895425

In [24]:
from sklearn.ensemble import *

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:591: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:600: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match

0.47549019607843135

In [25]:
clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:591: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:600: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match

0.4934640522875817

In [26]:
clf = AdaBoostClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:591: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:600: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match

0.5081699346405228

In [27]:
from sklearn.neighbors import *

clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:591: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:600: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match

0.4444444444444444

In [32]:
from hpsklearn import *
from hyperopt import *

clf = estim = HyperoptEstimator(classifier=any_classifier("my_clf"),
                              preprocessing=any_preprocessing("my_pre"),
                              algo=tpe.suggest,
                              max_evals=500,
                              trial_timeout=15)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

 50%|█████     | 1/2 [00:00<?, ?trial/s, best loss=?]

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


 92%|█████████▏| 12/13 [00:00<?, ?trial/s, best loss=?]

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


 94%|█████████▍| 17/18 [00:00<?, ?trial/s, best loss=?]

...

100%|█████████▉| 497/498 [00:00<?, ?trial/s, best loss=?]

/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


100%|██████████| 500/500 [00:03<00:00,  3.86s/trial, best loss: 0.46734693877551026]


/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:591: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match_prediction/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:600: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/maxlautenbach/PycharmProjects/football_match

0.5